In [1]:
## set R to search for libraries under my conda r-environemnt
.libPaths(c("/home/tbellagio/miniforge3/envs/r-environment/lib/R/library", .libPaths()))

In [2]:
.libPaths()

[1] "/home/tbellagio/miniforge3/envs/r-environment/lib/R/library"     
[2] "/home/tbellagio/miniforge3/envs/pipeline_snakemake/lib/R/library"

In [ ]:
path_ldp_af = '/carnegie/nobackup/scratch/xwu/grenet/merged_frequency/merged_hapFIRE_allele_frequency_LDpruned.txt'

allele_freq <- read.table(path_ldp_af, sep = '\t', header = TRUE, check.names = FALSE)
allele_freq <- allele_freq[, train_samples_vector]
allele_freq = t(allele_freq)

In [8]:
env <- read.csv('/carnegie/nobackup/scratch/tbellagio/gea_grene-net/leave_1_out/split_number_0/environment_lea.csv', header = TRUE, colClasses = "numeric")

In [9]:
env

bio1,bio12
<dbl>,<dbl>
-0.8192967,-0.1422553
-0.8192967,-0.1422553
-0.8192967,-0.1422553
-0.8192967,-0.1422553
-0.8192967,-0.1422553
-0.8192967,-0.1422553
-0.8192967,-0.1422553
-0.8192967,-0.1422553
-0.8192967,-0.1422553


In [10]:
env <- as.matrix(env)

In [11]:
library(data.table)
library(hash)
library(gradientForest)
library(raster)
library(rgeos)

Warning message:
“package ‘data.table’ was built under R version 4.3.2”
Warning message:
“package ‘hash’ was built under R version 4.3.0”
hash-3.0.1 provided by Decision Patterns



Attaching package: ‘hash’


The following object is masked from ‘package:data.table’:

    copy


Warning message:
“package ‘gradientForest’ was built under R version 4.3.2”
Warning message:
“package ‘raster’ was built under R version 4.3.1”
Loading required package: sp

Warning message:
“package ‘sp’ was built under R version 4.3.2”


ERROR: Error: package or namespace load failed for ‘raster’ in dyn.load(file, DLLpath = DLLpath, ...):
 unable to load shared object '/home/tbellagio/miniforge3/envs/r-environment/lib/R/library/terra/libs/terra.so':
  /home/tbellagio/miniforge3/envs/pipeline_snakemake/lib/R/bin/exec/../../.././libcrypto.so.3: version `OPENSSL_3.3.0' not found (required by /home/tbellagio/miniforge3/envs/r-environment/lib/R/library/terra/libs/../../../../././libssl.so.3)


In [ ]:
training_script = '''
#--------------------------------------------------------------------------------------------------------#
# Given a set of populations, allele freqs, and environmental data, train gradient forests.
#
# Usage
# -----
# Rscript gradient_training.R snpfile envfile range_file basename save_dir imports_path
#
# Notes
# -----
# - snpfile and envfile must already be subset for training populations
#
# Parameters
# ----------
# snpfile - SNPs that you want to use to train gradient forests (row.names=pop_ID, cols=loci)
# envfile - lat/long, environmental data for each pop
# range_file - tab delimited .txt file path with columns for `lat`, `lon`, and each env used in training
#    - this will be used to interpolate model on pops' lat/long to a larger area
#      - specifically this should the range-wide environmental data that is from same 
#        timeframe as in `envfile` (eg 1961-1990)
#    - generally, these are the envdata extracted from ClimateNA GIS data, and then clipped
#        to the range map
#    - these can be current, future, or uniform (eg for common garden) climate
#    - see eg yeaman03:/notebooks/007_JP_gea/07_gradient_forest/01_prep_spatial_envdata.ipynb
# basename - the basename prefix wanted for saving files in `save_dir`
# save_dir - the directory where files should be saved
# imports_path - path to directory with my homebrew R functions
#--------------------------------------------------------------------------------------------------------#

In [ ]:
dim(env) # 287*2

dim(allele_freq) # 287*13985

In [13]:
colnames(allele_freq) <- paste("Y", seq_len(ncol(allele_freq)), sep="")

In [34]:
colnames(allele_freq)

[1] "Y1"     "Y2"     "Y3"     "Y4"     "Y5"     "Y6"     "Y7"     "Y8"    
    [9] "Y9"     "Y10"    "Y11"    "Y12"    "Y13"    "Y14"    "Y15"    "Y16"   
   [17] "Y17"    "Y18"    "Y19"    "Y20"    "Y21"    "Y22"    "Y23"    "Y24"   
   [25] "Y25"    "Y26"    "Y27"    "Y28"    "Y29"    "Y30"    "Y31"    "Y32"   
   [33] "Y33"    "Y34"    "Y35"    "Y36"    "Y37"    "Y38"    "Y39"    "Y40"   
   [41] "Y41"    "Y42"    "Y43"    "Y44"    "Y45"    "Y46"    "Y47"    "Y48"   
   [49] "Y49"    "Y50"    "Y51"    "Y52"    "Y53"    "Y54"    "Y55"    "Y56"   
   [57] "Y57"    "Y58"    "Y59"    "Y60"    "Y61"    "Y62"    "Y63"    "Y64"   
   [65] "Y65"    "Y66"    "Y67"    "Y68"    "Y69"    "Y70"    "Y71"    "Y72"   
   [73] "Y73"    "Y74"    "Y75"    "Y76"    "Y77"    "Y78"    "Y79"    "Y80"   
   [81] "Y81"    "Y82"    "Y83"    "Y84"    "Y85"    "Y86"    "Y87"    "Y88"   
   [89] "Y89"    "Y90"    "Y91"    "Y92"    "Y93"    "Y94"    "Y95"    "Y96"   
   [97] "Y97"    "Y98"    "Y99"    "Y100"   "Y101"   "Y102"   "Y103"   "Y104"  
  [105] "Y105"   "Y106"   "Y107"   "Y108"   "Y109"   "Y110"   "Y111"   "Y112"  
  [113] "Y113"   "Y114"   "Y115"   "Y116"   "Y117"   "Y118"   "Y119"   "Y120"  
  [121] "Y121"   "Y122"   "Y123"   "Y124"   "Y125"   "Y126"   "Y127"   "Y128"  
  [129] "Y129"   "Y130"   "Y131"   "Y132"   "Y133"   "Y134"   "Y135"   "Y136"  
  [137] "Y137"   "Y138"   "Y139"   "Y140"   "Y141"   "Y142"   "Y143"   "Y144"  
  [145] "Y145"   "Y146"   "Y147"   "Y148"   "Y149"   "Y150"   "Y151"   "Y152"  
  [153] "Y153"   "Y154"   "Y155"   "Y156"   "Y157"   "Y158"   "Y159"   "Y160"  
  [161] "Y161"   "Y162"   "Y163"   "Y164"   "Y165"   "Y166"   "Y167"   "Y168"  
  [169] "Y169"   "Y170"   "Y171"   "Y172"   "Y173"   "Y174"   "Y175"   "Y176"  
  [177] "Y177"   "Y178"   "Y179"   "Y180"   "Y181"   "Y182"   "Y183"   "Y184"  
  [185] "Y185"   "Y186"   "Y187"   "Y188"   "Y189"   "Y190"   "Y191"   "Y192"  
  [193] "Y193"   "Y194"   "Y195"   "Y196"   "Y197"   "Y198"   "Y199"   "Y200"  
  [201] "Y201"   "Y202"   "Y203"   "Y204"   "Y205"   "Y206"   "Y207"   "Y208"  
  [209] "Y209"   "Y210"   "Y211"   "Y212"   "Y213"   "Y214"   "Y215"   "Y216"  
  [217] "Y217"   "Y218"   "Y219"   "Y220"   "Y221"   "Y222"   "Y223"   "Y224"  
  [225] "Y225"   "Y226"   "Y227"   "Y228"   "Y229"   "Y230"   "Y231"   "Y232"  
  [233] "Y233"   "Y234"   "Y235"   "Y236"   "Y237"   "Y238"   "Y239"   "Y240"  
  [241] "Y241"   "Y242"   "Y243"   "Y244"   "Y245"   "Y246"   "Y247"   "Y248"  
  [249] "Y249"   "Y250"   "Y251"   "Y252"   "Y253"   "Y254"   "Y255"   "Y256"  
  [257] "Y257"   "Y258"   "Y259"   "Y260"   "Y261"   "Y262"   "Y263"   "Y264"  
  [265] "Y265"   "Y266"   "Y267"   "Y268"   "Y269"   "Y270"   "Y271"   "Y272"  
  [273] "Y273"   "Y274"   "Y275"   "Y276"   "Y277"   "Y278"   "Y279"   "Y280"  
  [281] "Y281"   "Y282"   "Y283"   "Y284"   "Y285"   "Y286"   "Y287"   "Y288"  
  [289] "Y289"   "Y290"   "Y291"   "Y292"   "Y293"   "Y294"   "Y295"   "Y296"  
  [297] "Y297"   "Y298"   "Y299"   "Y300"   "Y301"   "Y302"   "Y303"   "Y304"  
  [305] "Y305"   "Y306"   "Y307"   "Y308"   "Y309"   "Y310"   "Y311"   "Y312"  
  [313] "Y313"   "Y314"   "Y315"   "Y316"   "Y317"   "Y318"   "Y319"   "Y320"  
  [321] "Y321"   "Y322"   "Y323"   "Y324"   "Y325"   "Y326"   "Y327"   "Y328"  
  [329] "Y329"   "Y330"   "Y331"   "Y332"   "Y333"   "Y334"   "Y335"   "Y336"  
  [337] "Y337"   "Y338"   "Y339"   "Y340"   "Y341"   "Y342"   "Y343"   "Y344"  
  [345] "Y345"   "Y346"   "Y347"   "Y348"   "Y349"   "Y350"   "Y351"   "Y352"  
  [353] "Y353"   "Y354"   "Y355"   "Y356"   "Y357"   "Y358"   "Y359"   "Y360"  
  [361] "Y361"   "Y362"   "Y363"   "Y364"   "Y365"   "Y366"   "Y367"   "Y368"  
  [369] "Y369"   "Y370"   "Y371"   "Y372"   "Y373"   "Y374"   "Y375"   "Y376"  
  [377] "Y377"   "Y378"   "Y379"   "Y380"   "Y381"   "Y382"   "Y383"   "Y384"  
  [385] "Y385"   "Y386"   "Y387"   "Y388"   "Y389"   "Y390"   "Y391"   "Y392"  
  [393] "Y393"   "Y394"   "Y395"   "Y396"   "Y397"   "Y398"   "Y399"   "Y400"  
  [4

In [14]:
env = as.data.frame(env)
allele_freq = as.data.frame(allele_freq)

In [28]:
cbind(env, allele_freq)

,bio1,bio12,Y1,Y2,Y3,Y4,Y5,Y6,Y7,Y8,⋯,Y13976,Y13977,Y13978,Y13979,Y13980,Y13981,Y13982,Y13983,Y13984,Y13985
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1_2_2,-0.8192967,-0.1422553,0.010930191,0.3862239,0.047550552,0.4681153,0.32902064,0.031522840,0.076735794,0.38262466,⋯,0.6557062,0.08572614,0.061988443,0.057968680,0.048042629,0.42181808,0.010114804,0.14068004,0.068164371,0.019793325
1_2_3,-0.8192967,-0.1422553,0.017849547,0.5092570,0.036267026,0.4076468,0.37055341,0.028921359,0.092680786,0.46095158,⋯,0.7172415,0.02355271,0.024033453,0.074933231,0.084424436,0.64194461,0.071371265,0.03418405,0.027096667,0.104908863
1_3_1,-0.8192967,-0.1422553,0.018318958,0.7862276,0.020203875,0.7929872,0.18067971,0.003820250,0.006643500,0.64345675,⋯,0.2424758,0.12427737,0.004067625,0.021422250,0.004742333,0.12594079,0.001230667,0.12684450,0.001685833,0.001728208
1_3_4,-0.8192967,-0.1422553,0.000251200,0.6519082,0.003278700,0.6218510,0.04178055,0.033424350,0.001889900,0.83916685,⋯,0.8915836,0.00122950,0.033381200,0.002020450,0.038966150,0.81700005,0.000628700,0.00133155,0.000219000,0.003050350
1_3_5,-0.8192967,-0.1422553,0.011536444,0.5170055,0.094492361,0.6456182,0.56332587,0.017954236,0.026488569,0.44785515,⋯,0.5349074,0.01596385,0.090579389,0.234912472,0.056017708,0.17191971,0.011160847,0.24769810,0.013674083,0.010511708
1_3_6,-0.8192967,-0.1422553,0.002585409,0.4666358,0.004475136,0.7803619,0.57088364,0.005456114,0.006403136,0.23839266,⋯,0.8662859,0.07688059,0.006583705,0.004527136,0.013038295,0.22604857,0.349496227,0.20447714,0.003448659,0.071368545
1_3_7,-0.8192967,-0.1422553,0.063799887,0.1603986,0.009004400,0.1802288,0.70149077,0.010202183,0.014028226,0.19317103,⋯,0.4563843,0.01768338,0.066861000,0.019174696,0.059357878,0.14879830,0.015712913,0.05384582,0.108491287,0.005763713
1_3_8,-0.8192967,-0.1422553,0.061102375,0.7213735,0.108839750,0.8034282,0.38305262,0.030004875,0.012880250,0.02899837,⋯,0.7907393,0.13358550,0.003681750,0.299987125,0.005238625,0.11076075,0.002526500,0.14862687,0.038475500,0.004307375
1_3_9,-0.8192967,-0.1422553,0.090899048,0.2593981,0.149177333,0.8035945,0.79052390,0.152855238,0.105056571,0.35280086,⋯,0.6536255,0.07176848,0.071335190,0.206625429,0.046219000,0.14697552,0.133157762,0.19757043,0.098267190,0.006517095


In [16]:
### TRAIN GRADIENT FORESTS

# NOTE: envdata below must have been filtered for env-only columns
# calc of maxLevel is different but not effectively from Ellis 2012 Supp 1 https://doi.org/10.6084/m9.figshare.c.3304350.v1
# recommended by Stephen Keller
maxLevel <- floor(log2(0.368*nrow(env)/2))
ptm <- proc.time()
gfOut <- gradientForest(cbind(env, allele_freq),
                        predictor.vars=colnames(env),
                        response.vars=colnames(allele_freq),
                        ntree=4,
                        transform=NULL,
                        nbin=201,  # this isn't doing anything since I didn't use `compact`
                        maxLevel=maxLevel,
                        corr.threshold=0.5)
print(proc.time() - ptm)


   user  system elapsed 
 40.589   0.310  41.115 


In [ ]:
predOut.gf <- predict(gfOut, env)

In [51]:
dim(predOut.gf) #287*2

[1] 287   2

In [37]:
importance(gfOut)

bio1      bio12 
0.04552471 0.04488390

In [47]:
gfOut

A forest of 4 regression trees for each of 4495 species

Call:

gradientForest(data = cbind(env, allele_freq), predictor.vars = colnames(env), 
    response.vars = colnames(allele_freq), ntree = 4, transform = NULL, 
    maxLevel = maxLevel, corr.threshold = 0.5, nbin = 201)



Important variables:
[1] bio1  bio12


bio1,bio12
<dbl>,<dbl>
-0.8192967,-0.1422553
-0.8192967,-0.1422553
-0.8192967,-0.1422553
-0.8192967,-0.1422553
-0.8192967,-0.1422553
-0.8192967,-0.1422553
-0.8192967,-0.1422553
-0.8192967,-0.1422553
-0.8192967,-0.1422553


In [49]:
predOut.gf <- predict(gfOut, env)

In [50]:
predOut.gf

bio1,bio12
<dbl>,<dbl>
0.01125208,0.011450777
0.01125208,0.011450777
0.01125208,0.011450777
0.01125208,0.011450777
0.01125208,0.011450777
0.01125208,0.011450777
0.01125208,0.011450777
0.01125208,0.011450777
0.01125208,0.011450777


In [40]:
env_pred <- read.csv('/carnegie/nobackup/scratch/tbellagio/gea_grene-net/leave_1_out/split_number_0/testing_environment_lea_full_genome.csv', header = TRUE, colClasses = "numeric")

In [43]:
env_pred[,1] = NULL

In [44]:
env_pred

bio1,bio12
<dbl>,<dbl>
0.8744907,-0.3862063
-2.0040338,1.8327043
-1.3790421,0.3353414
-0.5036921,-0.7964454
-0.6139786,0.3107879
-0.4358370,-0.2666404


In [26]:
predOut.gf <- predict(gfOut, env_pred)

In [45]:
predict(gfOut, newdata = env_pred)

bio1,bio12
<dbl>,<dbl>
0.033848967,0.009163176
-0.002945524,0.027846259
0.003889800,0.016112369
0.015146914,0.006210823
0.012984210,0.015799003
0.016101694,0.010480682


In [46]:
predict(gfOut)

,bio1,bio12
,<dbl>,<dbl>
1_2_2,0.01125208,0.011450777
1_2_3,0.01125208,0.011450777
1_3_1,0.01125208,0.011450777
1_3_4,0.01125208,0.011450777
1_3_5,0.01125208,0.011450777
1_3_6,0.01125208,0.011450777
1_3_7,0.01125208,0.011450777
1_3_8,0.01125208,0.011450777
1_3_9,0.01125208,0.011450777


In [27]:
> imp.vars <- names(importance(gf))
> Trns_grid <- cbind(Phys_grid[, c("EAST", "NORTH")],
+ predict(gf, Phys_grid[, imp.vars]))
It is useful to also transform the site environmental predictors, which are available from
gf$X.
> Trns_site <- predict(gf)

bio1,bio12
<dbl>,<dbl>
0.033848967,0.009163176
-0.002945524,0.027846259
0.003889800,0.016112369
0.015146914,0.006210823
0.012984210,0.015799003
0.016101694,0.010480682


In [ ]:
### INTEROPOLATE MODEL TO RANGE_WIDE DATA FOR THE SAME TIME PERIOD
cat(sprintf('\n\nInterpolating gradient forests model ...\n'))
ptm <- proc.time()
predOut.gf <- predict(gfOut, range_data[ ,envs])
print(proc.time() - ptm)



### SAVE
cat(sprintf('\n\nSaving files ...\n'))
gfOut_trainingfile <- paste0(
    save_dir,
    sprintf('/%s_gradient_forest_training.RDS', basename)
)
saveRDS(gfOut, gfOut_trainingfile)

predfile <- paste0(
    save_dir,
    sprintf('/%s_gradient_forest_predOut.RDS', basename)
)
saveRDS(predOut.gf, predfile)

print(gfOut_trainingfile)
print(predfile)


### END
cat(sprintf('\nDONE!\n'))
print(format(Sys.time(), format="%B %d %Y %T"))

'''

In [ ]:
gfOut

In [16]:
allele_freq.T

ERROR: Error in eval(expr, envir, enclos): object 'allele_freq.T' not found


In [ ]:



options(datatable.fread.datatable=FALSE)

args = commandArgs(trailingOnly=TRUE)
snpfile <- args[1]
envfile <- args[2]
range_file <- args[3]
basename <- args[4]
save_dir <- args[5]
imports_path <- args[6]

source(paste0(imports_path, '/imports.R'))  # https://github.com/brandonlind/r_imports

print(format(Sys.time(), format="%B %d %Y %T"))

prep_snps <- function(snpfile, exclude=c(), sep='\\t'){
    #----------------------------------------------------------------#
    # READ IN FREQUENCYxPOP TABLE, REMOVE POPS IN exclude
    #
    # Notes
    # -----
    # - assumes snpfile has colname 'index' which has pop names
    # - the rest of the colnames are names for each locus in dataset
    #----------------------------------------------------------------#
    # load data
    snps <- fread(snpfile, sep=sep)
    # set row names
    row.names(snps) <- snps[ ,'index']
    # remove column so all cols are loci
    snps <- subset(snps, select = -c(index))
    # get popnames, exclude irrelevant columns
    pops <- rownames(snps)[!rownames(snps) %in% c('CHROM', 'POS')]
    if (len(exclude) > 0){
        # exclude any other pops
        pops <- pops[!pops %in% exclude]
    }
    # reframe datatable
    snps <- snps[pops, ]
    return(snps)
}

prep_envdata <- function(envfile, pops){
    #-------------------------------------------------------------------------#
    # COLOR GEOGRAPHIC GROUPS AND REORDER POPS IN ENVIRONMENT TABLE 
    #
    # Notes
    # -----
    # - places `envs` (names of environments) in global namespace
    #-------------------------------------------------------------------------#
    
    # read in the environmental data (previously centered/standardized)
    envdata <- read.csv(envfile, sep='\\t', row.names=1)

    envs <- c('AHM', 'CMD', 'DD5', 'DD_0', 'EMT', 'EXT', 'Eref', 'FFP', 'MAP', 'MAT', 'MCMT', 'MSP', 'MWMT',
              'NFFD', 'PAS', 'SHM', 'TD', 'bFFP', 'eFFP')

    stopifnot(all(envs %in% colnames(envdata)))
    cat(sprintf('There are %s environmental attributes in the envdata table.', len(envs)))
    
    # order pops same as snp table
    envdata <- envdata[pops, ]
    
    # add to global namespace so I don't have to return a stupid list, stupid R
    envs <<- envs
    
    return(envdata[, envs])
}




### SET UP NAMESPACE

# get allele freqs for each pop
cat(sprintf('\nPrepping snp data ...\n'))
cand_freqs <- prep_snps(snpfile) #, exclude=test_pops)

# get population names
pops <- rownames(cand_freqs)
cat(sprintf('\nPops ...\n'))
cat(sprintf(pops))

# prep training data (climate data at location of populations)
cat(sprintf('\n\nPrepping envdata\n ...'))
envdata <- prep_envdata(envfile, pops)
range_data <- read.csv(range_file, sep='\t')




### TRAIN GRADIENT FORESTS

cat(sprintf('\n\nTraining gradient forests ...\n'))

# double check that pops are in same order
print(rownames(envdata))
print(rownames(cand_freqs))
stopifnot(all(rownames(envdata)==rownames(cand_freqs)))

# NOTE: envdata below must have been filtered for env-only columns
# calc of maxLevel is different but not effectively from Ellis 2012 Supp 1 https://doi.org/10.6084/m9.figshare.c.3304350.v1
# recommended by Stephen Keller
maxLevel <- floor(log2(0.368*nrow(envdata)/2))
ptm <- proc.time()
gfOut <- gradientForest(cbind(envdata, cand_freqs),
                        predictor.vars=colnames(envdata[ ,envs]),
                        response.vars=colnames(cand_freqs),
                        ntree=500,
                        transform=NULL,
                        nbin=201,  # this isn't doing anything since I didn't use `compact`
                        maxLevel=maxLevel,
                        corr.threshold=0.5)
print(proc.time() - ptm)




### INTEROPOLATE MODEL TO RANGE_WIDE DATA FOR THE SAME TIME PERIOD
cat(sprintf('\n\nInterpolating gradient forests model ...\n'))
ptm <- proc.time()
predOut.gf <- predict(gfOut, range_data[ ,envs])
print(proc.time() - ptm)



### SAVE
cat(sprintf('\n\nSaving files ...\n'))
gfOut_trainingfile <- paste0(
    save_dir,
    sprintf('/%s_gradient_forest_training.RDS', basename)
)
saveRDS(gfOut, gfOut_trainingfile)

predfile <- paste0(
    save_dir,
    sprintf('/%s_gradient_forest_predOut.RDS', basename)
)
saveRDS(predOut.gf, predfile)

print(gfOut_trainingfile)
print(predfile)


### END
cat(sprintf('\nDONE!\n'))
print(format(Sys.time(), format="%B %d %Y %T"))

'''

In [31]:
require(gradientForest)

In [32]:
load("GZ.sps.mat.Rdata")

Warning message in readChar(con, 5L, useBytes = TRUE):
“cannot open compressed file 'GZ.sps.mat.Rdata', probable reason 'No such file or directory'”


ERROR: Error in readChar(con, 5L, useBytes = TRUE): cannot open the connection


In [ ]:
dim(Sp_mat)

In [6]:
train_samples_vector = c('1_2_2',
  '1_2_3',
  '1_3_1',
  '1_3_4',
  '1_3_5',
  '1_3_6',
  '1_3_7',
  '1_3_8',
  '1_3_9',
  '1_3_10',
  '1_3_11',
  '1_3_12',
  '2_1_8',
  '2_1_9',
  '2_1_11',
  '2_1_12',
  '2_2_5',
  '2_2_6',
  '2_2_7',
  '5_3_1',
  '5_3_2',
  '5_3_3',
  '5_3_4',
  '5_3_5',
  '5_3_6',
  '5_3_7',
  '5_3_8',
  '5_3_9',
  '5_3_10',
  '5_3_11',
  '5_3_12',
  '6_1_1',
  '6_1_4',
  '6_2_2',
  '6_2_3',
  '6_2_5',
  '6_2_6',
  '6_2_7',
  '6_2_8',
  '6_2_9',
  '6_2_10',
  '6_2_11',
  '6_2_12',
  '9_3_1',
  '9_3_2',
  '9_3_3',
  '9_3_4',
  '9_3_5',
  '9_3_6',
  '9_3_7',
  '9_3_8',
  '9_3_9',
  '9_3_10',
  '9_3_11',
  '9_3_12',
  '10_1_3',
  '10_1_4',
  '10_1_6',
  '10_1_7',
  '10_1_11',
  '10_1_12',
  '10_2_1',
  '10_2_5',
  '10_2_8',
  '10_2_9',
  '10_3_2',
  '10_3_10',
  '11_1_4',
  '11_1_8',
  '11_1_9',
  '11_1_11',
  '11_2_2',
  '11_2_7',
  '11_2_12',
  '11_3_1',
  '11_3_3',
  '11_3_5',
  '11_3_6',
  '12_1_2',
  '12_1_3',
  '12_1_5',
  '12_1_6',
  '12_1_7',
  '12_1_8',
  '12_1_9',
  '12_2_4',
  '12_2_10',
  '12_2_11',
  '12_2_12',
  '13_1_6',
  '13_2_7',
  '13_3_1',
  '13_3_2',
  '13_3_3',
  '13_3_4',
  '13_3_5',
  '13_3_8',
  '13_3_9',
  '13_3_10',
  '13_3_11',
  '13_3_12',
  '23_1_1',
  '23_1_2',
  '23_1_3',
  '23_1_4',
  '23_1_5',
  '23_1_6',
  '23_1_7',
  '23_1_8',
  '23_1_9',
  '23_1_10',
  '23_1_11',
  '23_1_12',
  '25_2_1',
  '25_2_2',
  '25_2_3',
  '25_2_4',
  '25_2_5',
  '25_2_6',
  '25_2_7',
  '25_2_8',
  '25_2_9',
  '25_2_10',
  '25_2_11',
  '25_2_12',
  '26_1_1',
  '26_1_2',
  '26_1_3',
  '26_1_4',
  '26_1_5',
  '26_1_6',
  '26_1_8',
  '26_1_9',
  '26_1_10',
  '26_1_11',
  '26_1_12',
  '27_2_2',
  '27_2_3',
  '27_2_5',
  '27_3_1',
  '27_3_4',
  '27_3_6',
  '27_3_7',
  '27_3_8',
  '27_3_9',
  '27_3_10',
  '27_3_11',
  '27_3_12',
  '28_2_7',
  '28_2_8',
  '28_2_11',
  '28_3_2',
  '28_3_3',
  '28_3_4',
  '28_3_6',
  '28_3_12',
  '28_3_13',
  '28_3_15',
  '28_3_16',
  '28_3_17',
  '32_1_5',
  '32_1_11',
  '32_2_8',
  '32_2_9',
  '32_3_1',
  '32_3_2',
  '32_3_3',
  '32_3_4',
  '32_3_6',
  '32_3_7',
  '32_3_10',
  '32_3_12',
  '33_1_1',
  '33_2_4',
  '33_2_5',
  '33_2_6',
  '33_2_9',
  '33_2_11',
  '33_2_12',
  '37_1_1',
  '37_1_2',
  '37_1_3',
  '37_1_4',
  '37_1_5',
  '37_1_6',
  '37_1_7',
  '37_1_8',
  '37_1_9',
  '37_1_10',
  '37_1_11',
  '37_1_12',
  '42_1_2',
  '42_1_12',
  '42_2_11',
  '42_3_1',
  '42_3_3',
  '42_3_4',
  '42_3_5',
  '42_3_6',
  '42_3_7',
  '42_3_8',
  '42_3_9',
  '42_3_10',
  '43_1_3',
  '43_1_10',
  '43_2_1',
  '43_3_2',
  '43_3_4',
  '43_3_5',
  '43_3_6',
  '43_3_7',
  '43_3_8',
  '43_3_9',
  '43_3_11',
  '43_3_12',
  '45_2_3',
  '45_2_7',
  '45_3_1',
  '45_3_2',
  '45_3_4',
  '45_3_5',
  '45_3_6',
  '45_3_8',
  '45_3_9',
  '45_3_10',
  '45_3_11',
  '45_3_12',
  '46_3_1',
  '46_3_2',
  '46_3_3',
  '46_3_4',
  '46_3_5',
  '46_3_6',
  '46_3_7',
  '46_3_8',
  '46_3_9',
  '46_3_10',
  '46_3_11',
  '46_3_12',
  '49_2_1',
  '49_2_2',
  '49_2_3',
  '49_2_4',
  '49_2_5',
  '49_2_6',
  '49_2_7',
  '49_2_8',
  '49_2_9',
  '49_2_10',
  '49_2_11',
  '49_2_12',
  '53_3_1',
  '53_3_2',
  '53_3_3',
  '53_3_4',
  '53_3_5',
  '53_3_6',
  '53_3_7',
  '53_3_8',
  '53_3_9',
  '53_3_10',
  '53_3_11',
  '53_3_12',
  '54_3_1',
  '54_3_2',
  '54_3_3',
  '54_3_4',
  '54_3_5',
  '54_3_6',
  '54_3_7',
  '54_3_8',
  '54_3_9',
  '54_3_10',
  '54_3_11',
  '54_3_12',
  '60_1_1',
  '60_1_2',
  '60_1_3',
  '60_1_4',
  '60_1_5',
  '60_1_6',
  '60_1_7',
  '60_1_8',
  '60_1_9',
  '60_1_10',
  '60_1_11',
  '60_1_12')